In [1]:
def download_pic(people_name):
    # Open up web driver and goes to Google Images
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
    driver.get('https://www.google.ca/imghp?hl=en&tab=ri&authuser=0&ogbl')
    box = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')
    box.send_keys(people_name)
    box.send_keys(Keys.ENTER)
    img = driver.find_element_by_xpath('//*[@id="islrg"]/div[1]/div[1]/a[1]/div[1]/img')
    src = img.get_attribute('src')

    # download the image
    FILE_PATH = "../crawler/result/fit_UI/pic/" + people_name + ".png"
    pathlib.Path(FILE_PATH).parent.mkdir(parents=True, exist_ok=True)
    urllib.request.urlretrieve(src, FILE_PATH)
    return FILE_PATH

In [2]:

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import urllib
import pandas as pd
import numpy as np
import json
import pathlib
import ast
import argparse
from bertopic import BERTopic
import shutil

c:\Users\User\anaconda3\envs\kib\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def find_name_and_pic(name,person_unit_opinion_dict):
    for candidate in person_unit_opinion_dict.keys():
        # print(candidate)
        if name in candidate or candidate in name:
            return candidate, person_unit_opinion_dict[candidate]['pic_path']
    person_unit_opinion_dict[name] = {}
    person_unit_opinion_dict[name]['pic_path'] = 'pic_path' + name # download_pic(name)
    person_unit_opinion_dict[name]['opinions'] = {}
    return name, person_unit_opinion_dict[name]['pic_path']

In [5]:
# read current total file and person unit file
news_df = pd.read_csv('../result/total_opinions.csv')
topic_df = pd.read_csv('../result/topic_info.csv')

# 將放上 news_df # 有用嗎?
represent_title = []
represent_index_list = []
for i in range(len(news_df)):
    topic_represent_article_index = topic_df.iloc[news_df.iloc[i]['topic']]['topic_represent_article']
    represent_index_list.append(topic_represent_article_index)
    represent_title.append(news_df.iloc[topic_represent_article_index]['title'])
news_df['represent_index'] = represent_index_list 
news_df['represent_title'] = represent_title

In [54]:
# read cluster_cluster dataframe
CCPATH = 'cluster_cluster_table.csv'
try:
    cluster_cluster_df = pd.read_csv(CCPATH)
except:
    print("create empty cluster_cluster_df ")
    cluster_cluster_df = pd.DataFrame(columns = ['little_cluster_title', 'little_cluster_number', 'date', 'total_cluster_title', 'total_cluster_number'])

In [55]:
# take today time for ccdf
import datetime
yesterday = datetime.date.today() - datetime.timedelta(days=1)
yesterday_str = yesterday.strftime("%Y-%m-%d")

In [56]:
# append new little cluster form news_df and topic
for i in range(len(topic_df)):
    little_cluster = topic_df.iloc[i]['topic_index']
    represent_title = news_df.iloc[topic_df.iloc[i]['topic_represent_article']]['title']
    temp_df = pd.DataFrame({'little_cluster_title':represent_title, 'little_cluster_number':little_cluster, 'date':yesterday_str, 'total_cluster_title':np.nan, 'total_cluster_number':np.nan}, index=[0])
    cluster_cluster_df = cluster_cluster_df.append(temp_df, ignore_index=True)

In [57]:
cluster_cluster_df

,little_cluster_title,little_cluster_number,date,total_cluster_title,total_cluster_number
0,為爭選桃園市長失業 羅智強被「突襲」後宣布下一步,0,2022-05-24,NaN,NaN
1,不應該被排除!蔡英文:台灣需要WHA WHA也需要台灣,1,2022-05-24,NaN,NaN


In [58]:
# knn
def cluster_article(article_list):
    if len(article_list) == 2:
        return np.array([0,1]), np.array([0,1])
    from sentence_transformers import SentenceTransformer
    from sklearn.cluster import KMeans
    from sklearn.metrics import pairwise_distances_argmin_min, silhouette_score
    embedder = SentenceTransformer('ckiplab/bert-base-chinese')
    corpus_embeddings = embedder.encode(article_list)  
    # Find out the best score
    sil_coeff_list = []
    least_cluster_num = 0 if int(len(corpus_embeddings)/25) > 1 else 2
    max_cluster_num = 2 if int(len(corpus_embeddings)/10) <= 2 else int(len(corpus_embeddings)/10)
    if least_cluster_num == max_cluster_num:
        max_cluster_num += 1
    for n_cluster in range(least_cluster_num,max_cluster_num):
        clustering_model = KMeans(n_clusters=n_cluster).fit(corpus_embeddings)
        label = clustering_model.labels_
        sil_coeff = silhouette_score(corpus_embeddings, label, metric='euclidean')
        sil_coeff_list.append(sil_coeff)

    # Get best clusters
    num_clusters = sil_coeff_list.index(max(sil_coeff_list)) + least_cluster_num

    # Perform k-means clustering
    clustering_model = KMeans(n_clusters=num_clusters)
    clustering_model.fit(corpus_embeddings)
    cluster_label_list = clustering_model.labels_

    # Find the closet sentence in every cluster to represent group meaning
    closest_index_list, _ = pairwise_distances_argmin_min(clustering_model.cluster_centers_, corpus_embeddings)
    
    return cluster_label_list, closest_index_list

In [62]:
# implement knn
topic_index_list, represent_index_list =cluster_article(cluster_cluster_df['little_cluster_title'])

In [64]:
# add total cluster info to ccdf
total_cluster_title_list = []
for i in range(len(topic_index_list)):
    temp_represent_index =  represent_index_list[topic_index_list[i]]
    total_cluster_title_list.append(cluster_cluster_df.iloc[temp_represent_index]['little_cluster_title'])
cluster_cluster_df['total_cluster_title'] = total_cluster_title_list
cluster_cluster_df['total_cluster_number'] = topic_index_list

In [65]:
cluster_cluster_df

,little_cluster_title,little_cluster_number,date,total_cluster_title,total_cluster_number
0,為爭選桃園市長失業 羅智強被「突襲」後宣布下一步,0,2022-05-24,為爭選桃園市長失業 羅智強被「突襲」後宣布下一步,0
1,不應該被排除!蔡英文:台灣需要WHA WHA也需要台灣,1,2022-05-24,不應該被排除!蔡英文:台灣需要WHA WHA也需要台灣,1


In [66]:
try:
    with open('all_person.json',encoding="utf-8") as json_file:
        person_unit_opinion_dict = json.load(json_file)
except:
    print("create empty person unit opinion")
    person_unit_opinion_dict = {}

# create pic image link
for i in range(len(news_df)):
    opinions_list = ast.literal_eval(news_df.iloc[i]["opinions_list"])
    for opinions in opinions_list:
        for opinions_item in opinions[0]:
            label = opinions_item['labels']
            text = opinions_item['text']
            if label == ['person']:
                find_name_and_pic(text, person_unit_opinion_dict)

# change all_opinions name
for i in range(len(news_df)):
    opinions_list = ast.literal_eval(news_df.iloc[i]["all_opinions_after_cluster"])
    for j in range(len(opinions_list)):
        # find person row
        # add opinions including time stamp
        opinions_list[j]['person'],_ = find_name_and_pic(opinions_list[j]['person'],person_unit_opinion_dict)
    news_df.iloc[i, news_df.columns.get_loc('all_opinions_after_cluster')] = str(opinions_list)

create empty person unit opinion


In [ ]:
person_unit_opinion_dict

In [68]:
news_df['represent_title'].unique()

array(['為爭選桃園市長失業 羅智強被「突襲」後宣布下一步', '不應該被排除!蔡英文:台灣需要WHA WHA也需要台灣'],
      dtype=object)

In [69]:
# add person new opinions 
for i in range(len(news_df)):
    represent_title = news_df.iloc[i]['represent_title']
    opinions_list = ast.literal_eval(news_df.iloc[i]["all_opinions_after_cluster"])
    for opinion in opinions_list:
        opinion['datetime'] = news_df.iloc[i]['datetime']
        del opinion['order']
        # find person row
        # add opinions including time stamp
        name,_ = find_name_and_pic(opinion['person'],person_unit_opinion_dict)
        opinion['person'] = name

        if represent_title not in person_unit_opinion_dict[name]['opinions']:
            person_unit_opinion_dict[name]['opinions'][represent_title] = []

        person_unit_opinion_dict[name]['opinions'][represent_title].append(opinion)

In [46]:
person_unit_opinion_dict

{'侯友宜': {'pic_path': 'pic_path侯友宜',
  'opinions': {'為爭選桃園市長失業 羅智強被「突襲」後宣布下一步': [{'person': '侯友宜',
     'verb': '指出',
     'opinion': '他的整體聲量高點卻是落在4月19日(2.4萬),侯友宜因新北兩歲童等不到救護車離世一案,遭到質疑。',
     'datetime': '2022-05-22 21:55'},
    {'person': '侯友宜',
     'verb': '表示',
     'opinion': '「我知道總統太忙了,我會一肩承擔新北市的大小事,一定守好新北市!」。',
     'datetime': '2022-05-22 21:52'},
    {'person': '侯友宜',
     'verb': '指出',
     'opinion': '總統到了這個星期四才打了電話給新北市基層醫師,「我想總統可能忙到不知道,您還沒致電羅醫師的快一個月前,新北市早在4/28已經邀請了新北市620家的基層診所,共同啟動居家照護第三階段計畫,投入居家照護以及遠距醫療,到現在基層診所派案每天達到六千人,已經達到居家照護數的35%。」',
     'datetime': '2022-05-22 18:58'},
    {'person': '侯友宜',
     'verb': '表示',
     'opinion': '從這些地方,就可以知道新北基層診所的醫護和市府團隊並肩作戰,熱血投入,守護著新北市,「也讓我們得以保留醫療量能,守住中重症病患!」侯友宜指出,總統沒空,新北在疫情最前線,新北市府依舊自立自強,保全醫療量能,訂定解決之道,全力以赴。',
     'datetime': '2022-05-22 18:58'},
    {'person': '侯友宜',
     'verb': '表示',
     'opinion': '已經向中央反映一個月以上,關鍵須解決中央系統問題,實在忍無可忍。',
     'datetime': '2022-05-22 17:04'},
    {'person': '侯友宜',
     'verb': '質疑',
     'opinion': '為何不

In [70]:
news_df.head(1)

,title,article,url,company,category,reporter,space_index,datetime,year,month,...,minute,article_coref,people_name,ner,topic,representative_docs,opinions_list,all_opinions_after_cluster,represent_index,represent_title
0,綠營大數據「疫情影響趨穩」 藍營聲量聚焦桃園內亂、徐巧芯違停,"在本土疫情升溫之際,在野黨持續針對快篩、用藥等議題砲轟執政團隊,不過在快篩劑鋪貨逐漸充足後,...",https://www.ettoday.net/news/20220522/2256707.htm,東森,政治,陶本和,"[118, 224, 339, 432, 491, 593, 707, 793, 871, ...",2022-05-22 21:55,2022,5,...,55,"在本土疫情升溫之際,在野黨持續針對快篩、用藥等議題砲轟執政團隊,不過在快篩劑鋪貨逐漸充足後,...","['張善政', '蔣萬安', '侯友宜', '康復者聯盟', '柯文哲', '徐巧芯', '...","['桃園', '張善政', '台北市', '蔣萬安', '侯友宜', '康復者聯盟', '柯...",0,0,"[[[{'start': 876, 'end': 879, 'text': '侯友宜', '...","[{'person': '侯友宜', 'verb': '指出', 'opinion': '他...",42,為爭選桃園市長失業 羅智強被「突襲」後宣布下一步


In [71]:
PERSON_SLICE_FILE_PATH = 'cluster_cluster_table.csv'
cluster_cluster_df.to_csv(PERSON_SLICE_FILE_PATH, index=False)

In [72]:
# slice the json to cluster_cluster json
with open('all_person.json', 'w', encoding='utf-8') as fp:
    json.dump(person_unit_opinion_dict, fp,  indent=4, ensure_ascii=False)

In [73]:
with open('all_person.json',encoding="utf-8") as json_file:
    person_unit_opinion_dict = json.load(json_file)

In [74]:
cluster_cluster_df.head(1)

,little_cluster_title,little_cluster_number,date,total_cluster_title,total_cluster_number
0,為爭選桃園市長失業 羅智強被「突襲」後宣布下一步,0,2022-05-24,為爭選桃園市長失業 羅智強被「突襲」後宣布下一步,0


In [76]:
# create a new sliced fit UI result
if pathlib.Path('pubc_dic').is_dir():
    shutil.rmtree('pubc_dir')
for name in person_unit_opinion_dict:
    for small_cluster in person_unit_opinion_dict[name]['opinions']:
        # find total cluster
        for cc_index in range(len(cluster_cluster_df)):
            if cluster_cluster_df.iloc[cc_index]['little_cluster_title'] == small_cluster:
                total_cluster_title = cluster_cluster_df.iloc[cc_index]['total_cluster_title']
                total_cluster_number = cluster_cluster_df.iloc[cc_index]['total_cluster_number']
        # look this json create or not
        person_unit_big_cluster_name = 'pubc_dir/' + name + '-' + str(total_cluster_number) + '.json'
        try:
            with open(person_unit_big_cluster_name ,encoding="utf-8") as json_file:
                pubc_dict = json.load(json_file)
        except:
            # print("create empty person unit opinion")
            pubc_dict = {'pict_path':person_unit_opinion_dict[name]['pic_path'], 'opinions':[]}

        for opinions in person_unit_opinion_dict[name]['opinions'][small_cluster]:
            pubc_dict['opinions'].append(opinions)
        if len(pubc_dict['opinions']) == 0:
            print(name, small_cluster)
        # save file
        pathlib.Path(person_unit_big_cluster_name).parent.mkdir(parents=True, exist_ok=True)
        with open(person_unit_big_cluster_name, 'w', encoding='utf-8') as fp:
            json.dump(pubc_dict, fp,  indent=4, ensure_ascii=False)
